In [ ]:
import glob
import os
import iris
from iris.time import PartialDateTime
from iris.experimental.equalise_cubes import equalise_attributes

DATADIR = '/project/ciid/projects/PRECIS/worksheets/data_v2/EAS-22/'
# write outputs to own directory so that can be checked before upload under ciid
OUTDIR = f'{os.getenv("SCRATCH")}/cordex_inputs/'


# Make data for worksheet 4

In [ ]:
# Prep future data (2021-2050) from pp files

# Make future 2021-2050 data
for runid in ['cahpa', 'cahpb']:
    ppdir = os.path.join(DATADIR, 'pp', runid)

    # find all the files from which to remove the rim
    flist = glob.glob(ppdir + '/*pm[mnop]*.pp')
    
    datacubes = iris.load(flist)
    tempcubelist = []
    for cube in datacubes:
        temprimcubelist = []
        # In case the data are more than 2 dimensional, grab a 2d slice
        for yx_slice in cube.slices(['grid_latitude','grid_longitude']):
            norimcube = yx_slice[8 : -8 , 8 : -8]
            temprimcubelist.append(norimcube)
        # Merge the individual 2d slices into a single cube
        trimmedcube = iris.cube.CubeList(temprimcubelist).merge_cube()
        tempcubelist.append(trimmedcube)
    
    for cube in tempcubelist:
        if cube.name() == 'air_temperature':
            cube.convert_units('celsius')
            outpath = os.path.join(DATADIR, 'future', runid + '.mon.2021_2050.tm.rr.C.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            cube.convert_units('kg m-2 day-1')
            cube.units = 'mm day-1'
            outpath = os.path.join(DATADIR, 'future', runid + '.mon.2021_2050.pr.rr.mmday-1.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
            iris.save(cube, outpath)

In [ ]:
# Prep baseline data (1961-1990) from pp files

# Make baseline data
for runid in ['cahpa', 'cahpb']:
    ppdir = os.path.join(DATADIR, 'pp', runid)

    # find all the files from which to remove the rim
    flist = glob.glob(ppdir + '/*pm[ghij]*.pp')
    
    datacubes = iris.load(flist)
    tempcubelist = []
    for cube in datacubes:
        temprimcubelist = []
        # In case the data are more than 2 dimensional, grab a 2d slice
        for yx_slice in cube.slices(['grid_latitude','grid_longitude']):
            norimcube = yx_slice[8 : -8 , 8 : -8]
            temprimcubelist.append(norimcube)
        # Merge the individual 2d slices into a single cube
        trimmedcube = iris.cube.CubeList(temprimcubelist).merge_cube()
        tempcubelist.append(trimmedcube)
    
    for cube in tempcubelist:
        if cube.name() == 'air_temperature':
            cube.convert_units('celsius')
            outpath = os.path.join(DATADIR, 'historical', runid + '.mon.1961_1990.tm.rr.C.nc')
            cube.remove_coord('forecast_period')
            cube.remove_coord('forecast_reference_time')
            iris.save(cube, outpath)
        if cube.name() == 'precipitation_flux':
            pass

# Make data for Worksheet 5

The files needed are daily precipitation, with the two periods (historical and future) consolidated into a single file

Note that these datasets should be saved with compression to reduce their file size.  If for some reason this needs to be done during a PRECIS course, it is quicker to save them without compression using `iris.save(cube, outfile)` but the filesize is significantly larger.

Unlike the previous PRECIS data, the memory requirements of this code are modest (probably due to the lack of rim removal meaning the data remains lazy). It does take several minutes (~5) to run each example.

In [ ]:
# Prep future data (2041-2060) from CORDEX files

shortnames = {'MOHC-HadGEM2-ES':'hadgem2-es', 'MPI-M-MPI-ESM-LR':'mpi-esm-lr'}

# Make future 2041-2060 data
future_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=2041) 
                            <= cell.point <= PartialDateTime(year=2060))
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]

    # find all the files 
    flist = glob.glob(DATADIR + f'pr_EAS-22_{gcmid}_rcp85_*_GERICS-REMO2015_*_day_*.nc')
    print (f'input files: {flist}')
    
    # load using time constraint above
    datacubes = iris.load(flist, future_time_constraint)

    # merge the cubes
    equalise_attributes(datacubes)
    cube = datacubes.concatenate_cube()
    
    if cube.name() == 'air_temperature':
        cube.convert_units('celsius')
        outpath = os.path.join(OUTDIR, 'future', shortgcm + '.day.2041_2060.GERICS-REMO2015.tm.C.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    if cube.name() == 'precipitation_flux':
        cube.convert_units('kg m-2 day-1')
        cube.units = 'mm day-1'
        outpath = os.path.join(OUTDIR, 'future', shortgcm + '.day.2041_2060.GERICS-REMO2015.pr.mmday-1.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    print(f'saved file: {outpath}')


In [ ]:
# Prep baseline data (1986-2006) from CORDEX files
historical_time_constraint = iris.Constraint(time=lambda cell: PartialDateTime(year=1986) 
                            <= cell.point <= PartialDateTime(year=2005))
# Make baseline data
for gcmid in ['MOHC-HadGEM2-ES', 'MPI-M-MPI-ESM-LR']:

    # set a short name consistent with what used in the exercises [would it be better to use full names?]
    shortgcm = shortnames[gcmid]

    # find all the files 
    flist = glob.glob(DATADIR + f'pr_EAS-22_{gcmid}_historical_*_GERICS-REMO2015_*_day_*.nc')
    print (f'input files: {flist}')
    
    # load using time constraint above
    datacubes = iris.load(flist, historical_time_constraint)

    # merge the cubes
    equalise_attributes(datacubes)
    cube = datacubes.concatenate_cube()
    
    if cube.name() == 'air_temperature':
        cube.convert_units('celsius')
        outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.day.1986_2005.GERICS-REMO2015.tm.C.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    if cube.name() == 'precipitation_flux':
        cube.convert_units('kg m-2 day-1')
        cube.units = 'mm day-1'
        outpath = os.path.join(OUTDIR, 'historical', shortgcm + '.day.1986_2005.GERICS-REMO2015.pr.mmday-1.nc')
        iris.save(cube, outpath, least_significant_digit=3, 
                  complevel=4, zlib=True)
    print(f'saved file: {outpath}')

